Коллокации - это устойчивые выражения, состоящие из двух и более слов. Устойчивые - значит, что они часто используются вместе. Также часто значения коллокации не могут быть выведены лишь из значений, входящих в них слов.

In [ ]:
import itertools
from collections import Counter, defaultdict
import numpy as np
import re
from string import punctuation
from nltk.corpus import stopwords
from pymorphy3 import MorphAnalyzer
morph = MorphAnalyzer()
from nltk import word_tokenize
from nltk import sent_tokenize

Зададим несколько функций, которые будут обрабатывать наши тексты для большей объективности и чистоты: очистим от стоп-слов и лемматизируем. 
Функция **normalize** лемматизирует наш текст иоставит от них только кириллические слова и числа, приведет текст к нижнему регистру, а затем выкинет все слова короче 3 букв.  
Также нам нужен **sent_tokenize** из nltk.
**ngrammer** cобирает нграммы (заданной величины) и их частоты.

In [ ]:
stops = set(stopwords.words('russian') + ["это", "весь"]) 

def normalize(text):
    normalized_text = [morph.parse(token)[0].normal_form for token in word_tokenize(text.lower()) if len(token) > 2 and token not in stops]  
    return normalized_text

def ngrammer(tokens, n=2):
    ngrams = []
    tokens_cl = [token for token in word_tokenize(tokens.lower()) if token not in stops]
    for i in range(0, len(tokens_cl)-n+1):
        ngrams.append(tuple(tokens_cl[i:i+n]))
    return ngrams

Результат работы функции **normalize**:

In [ ]:
normalize('Я люблю кошек и собак. Я люблю фикусы и кактусы')

Результат **sent_tokenize**:

In [ ]:
list(sent_tokenize('Я люблю кошек и собак. Я люблю фикусы и кактусы'))

Результат **ngrammer**:

In [ ]:
ngrammer('Я люблю кошек и собак. Я люблю фикусы и кактусы')

Откроем письма Достоевского, разобъем на предложения и предобработаем каждое предложение:

In [ ]:
path = 'DostoyevskiEpistola.txt'
with open(path, encoding='utf-8') as txt:
    corpus = txt.read()
    corpus = re.sub(r'\n', ' ', corpus)
    corpus = re.sub(r'[a-zA-Z…]',  '', corpus)
    corpus_sent = sent_tokenize(corpus) #разбиение на предложения
    corpus_sent_clean = [] #cюда мы сохраняем чистые предложения
    for sent in corpus_sent: #пройдемся по каждому предложению
        corpus_sent_clean.append(' '.join(normalize(sent))) #предобработаем
    

In [ ]:
corpus[:100]

In [ ]:
corpus_sent_clean[:100]

Получим с помощью счётчика все биграммы с их частотами:

In [ ]:
word_counter = Counter()

for sent in corpus_sent_clean:
    word_counter.update(ngrammer(sent, n=2))
word_counter

In [ ]:
word_counter.most_common(15)

In [ ]:
type(word_counter)

В списке есть биграммы, которые попали в список из-за того, что одно слово очень частотное и вообще встречается много в каких контекстах. Нас скорее интересуют случаи, когда слова в большинстве случаев встречаются вместе. Для этого мы можем придумать какие-нибудь формулы, учитывающие частоты слов по отдельности и общую частоту.  

Самый простой способ - взять количество упоминаний биграма и поделить на сумму количеств упоминаний слов по отдельности.  

Такая формула называется PMI.

In [ ]:
def scorer_simple(word_count_a, word_count_b, bigram_count, *args): #*args позволяет подать в функцию любое количество аргументов
    try:
        score = bigram_count / (word_count_a + word_count_b)
    
    except ZeroDivisionError:
        return 0
    
    return score

#Сделаем функцию, которая будет делать счетчики для слов и биграммов.
def collect_stats(corpus, stops):
    ## соберем статистики для отдельных слов
    ## и биграммов
    
    unigrams = Counter()
    bigrams = Counter()
    
    for sent in corpus:
        unigrams.update(word_tokenize(sent))
        bigrams.update(ngrammer(sent))
    
    return unigrams, bigrams

#И функцию, которая пройдет по всем биграммам и вычислит для них нашу метрику.
def score_bigrams(unigrams, bigrams, scorer, threshold=-100000, min_count=1):
    ## посчитаем метрику для каждого нграмма
    bigram2score = Counter()
    len_vocab = len(unigrams)
    for bigram in bigrams:
        score = scorer(unigrams[bigram[0]], unigrams[bigram[1]], 
                       bigrams[bigram], len_vocab, min_count)
        
        ## если метрика выше порога, добавляем в словарик
        if score > threshold:
            bigram2score[bigram] = score
    
    return bigram2score

In [ ]:
unigrams, bigrams = collect_stats(corpus_sent_clean, stops)

In [ ]:
#print(unigrams)
#print(bigrams)

In [ ]:
bigram2score = score_bigrams(unigrams, bigrams, scorer_simple)
bigram2score

Проблема с таким подходом в том, что на самом верху окажутся слова, которые встречают по одному разу.

In [ ]:
bigram2score.most_common(15)

Поэтому можно немного переделать оценивающую функцию, добавив минимальное число вхождений для биграмма.

По этой ссылке можно прочитать про другие метрики.  
 
http://www.scielo.org.mx/scielo.php?script=sci_arttext&pid=S1405-55462016000300327#t1

**Все готовое**  

Писать все это самому не обязательно.

Самый удобный способ - использовать nltk. Тут больше метрик, но преборазователь слов в нграммы нужно написать самому.

In [ ]:
import nltk
from nltk.collocations import *

Список названий коллокационных мер, доступных в nltk находится здесь (таблица Methods): https://tedboy.github.io/nlps/generated/generated/nltk.BigramAssocMeasures.html

In [ ]:
corpus = ' '.join(corpus_sent_clean)
corpus[:1000]

In [ ]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
trigram_measures = nltk.collocations.TrigramAssocMeasures()

In [ ]:
finder_big = BigramCollocationFinder.from_words(word_tokenize(corpus))

In [ ]:
finder_trig = TrigramCollocationFinder.from_words(word_tokenize(corpus))

In [ ]:
finder_big

In [ ]:
finder_big.nbest(bigram_measures.raw_freq, 50)

Получим значения:

In [ ]:
scores = finder_big.score_ngrams(bigram_measures.chi_sq)
scores

Отсортируем в обратном порядке:

In [ ]:
sorted_scores = sorted(scores, key=lambda score: score[1], reverse=False)
sorted_scores

In [ ]:
finder_trig.nbest(trigram_measures.pmi, 15)

In [ ]:
scores = finder_trig.score_ngrams(trigram_measures.pmi)
scores

https://www.nltk.org/howto/collocations.html - дополнительные опции можно увидеть здесь.

Можно написать взвешенную меру.

In [ ]:
text = corpus[:100000]
text

In [ ]:
finder_big = BigramCollocationFinder.from_words(word_tokenize(text))
scores_lhr = finder_big.score_ngrams(bigram_measures.likelihood_ratio)
scores_raw = finder_big.score_ngrams(bigram_measures.student_t)
scores_pmi = finder_big.score_ngrams(bigram_measures.pmi)

Подпишем ранг у каждой из мер:

In [ ]:
rank_lhr = []
counter = 1
for colloc in scores_lhr:
    rank_lhr.append((colloc[0], counter))
    counter += 1

rank_lhr

In [ ]:
rank_raw = []
counter = 1
for colloc in scores_raw:
    rank_raw.append((colloc[0], counter))
    counter += 1

rank_pmi = []
counter = 1
for colloc in scores_pmi:
    rank_pmi.append((colloc[0], counter))
    counter += 1

In [ ]:
rank_raw

In [ ]:
rank_pmi

In [ ]:
all_ranks = [rank_lhr, rank_raw, rank_pmi]
all_ranks

In [ ]:
sum_of_ranks = {}
for bigs_ranks in all_ranks:
   # print(bigs_ranks)
    bigs, ranks = zip(*bigs_ranks) #звездочка разделяет кортежи
    #print(bigs)
    for index in range(len(ranks)):
        #print(index)
        if bigs[index] not in sum_of_ranks:
            sum_of_ranks[bigs[index]] = 0
        sum_of_ranks[bigs[index]] += ranks[index]

sum_of_ranks

Можно сделать списком:

In [ ]:
sorted_list = list(sum_of_ranks.items())
sorted(sorted_list, key=lambda sorted_list: sorted_list[1], reverse=False)

**Задание 1.** Выберите любой достаточно большой текст. Посчитайте коллокации (2-, 3-, 4- - любые) на нем с помощью любых трех мер.

**Задание 2.** Теперь посчитайте сумму рангов. Какие коллокации взвешенно наиболее весомые в вашем тексте?

**NB!** Коллокации можно извлекать не только по стоящим непосредственно друг перед другом токенами. Можно также указывать окно, в котором будет проводиться поиск коллокации.

In [ ]:
finder_big_win = BigramCollocationFinder.from_words(word_tokenize(text), window_size=5)

In [ ]:
scores_tscore_no_win = finder_big.score_ngrams(bigram_measures.student_t)
scores_tscore_win = finder_big_win.score_ngrams(bigram_measures.student_t)

In [ ]:
print(scores_tscore_no_win[:10])

In [ ]:
print(scores_tscore_win[:10])

Как видно, коллокации отличаются.

**Задание 3**. Посчитайте метрики на разных окнах для текста из заданий 1-2.

Теперь учтём частеречную разметку.

In [ ]:
corpus = ' '.join(corpus_sent_clean)
text = corpus
tagged_tuples = ['_'.join((str(morph.parse(token)[0].normal_form), str(morph.parse(token)[0].tag.POS)) for token in word_tokenize(text)]
tagged_text = ' '.join(tagged_tuples)
tagged_text[:1000]

In [ ]:
tagged_tuples

In [ ]:
finder_big = BigramCollocationFinder.from_words(word_tokenize(tagged_text))
scores_student_t = finder_big.score_ngrams(bigram_measures.student_t)

In [ ]:
scores_student_t

Теперь найдем все биграммы, в которых первое слово - существительное с помощью регулярки.

In [ ]:
import re

In [ ]:
bigrams_with_nouns = []
for bigram_measured in scores_student_t:
    #print(bigram_measured)
    #print(bigram_measured[0][0])
    if re.match(r'\w+_NOUN', bigram_measured[0][0]):
        #print(bigram_measured[0][0])
       # print(bigram_measured)
        bigrams_with_nouns.append(bigram_measured)
print(bigrams_with_nouns[:10])

**Задание 4.** Как переписать код выше, чтобы он искал биграммы, в которых второе слово - существительное, а первое любое?

**Задание 5.** Как переписать код выше, чтобы он искал биграммы, в которых первое слово прилагательное, а второе слово - существительное.